In [1]:
import os
import random
import gc

import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import r2_score

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from PIL import Image

from transformers import AutoImageProcessor, AutoModel
import os
import wandb

print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

2025-12-07 14:24:00.673840: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765117440.861313      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765117440.916631      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

Torch: 2.6.0+cu124
CUDA available: True


In [2]:
DINO_MODEL_DIR = "/kaggle/input/dinov2/pytorch/base/1"

In [3]:
os.environ["WANDB_MODE"] = "offline"

In [4]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(1488)

In [5]:
DATA_DIR = "/kaggle/input/csiro-biomass" 

train_path = os.path.join(DATA_DIR, "train.csv")
test_path  = os.path.join(DATA_DIR, "test.csv")

train_long = pd.read_csv(train_path)
test_df    = pd.read_csv(test_path)

train_long.head()

,sample_id,image_path,Sampling_Date,State,Species,Pre_GSHH_NDVI,Height_Ave_cm,target_name,target
0,ID1011485656__Dry_Clover_g,train/ID1011485656.jpg,2015/9/4,Tas,Ryegrass_Clover,0.62,4.6667,Dry_Clover_g,0.0000
1,ID1011485656__Dry_Dead_g,train/ID1011485656.jpg,2015/9/4,Tas,Ryegrass_Clover,0.62,4.6667,Dry_Dead_g,31.9984
2,ID1011485656__Dry_Green_g,train/ID1011485656.jpg,2015/9/4,Tas,Ryegrass_Clover,0.62,4.6667,Dry_Green_g,16.2751
3,ID1011485656__Dry_Total_g,train/ID1011485656.jpg,2015/9/4,Tas,Ryegrass_Clover,0.62,4.6667,Dry_Total_g,48.2735
4,ID1011485656__GDM_g,train/ID1011485656.jpg,2015/9/4,Tas,Ryegrass_Clover,0.62,4.6667,GDM_g,16.2750


In [6]:
TARGET_NAMES = [
    "Dry_Green_g",
    "Dry_Dead_g",
    "Dry_Clover_g",
    "GDM_g",
    "Dry_Total_g",
]

In [7]:
train_df = (
    train_long
    .pivot_table(
        index=[
            "image_path",
            "Sampling_Date",
            "State",
            "Species",
            "Pre_GSHH_NDVI",
            "Height_Ave_cm",
        ],
        columns="target_name",
        values="target",
        aggfunc="first"
    )
    .reset_index()
)

In [8]:
train_df.columns.name = None
train_df

,image_path,Sampling_Date,State,Species,Pre_GSHH_NDVI,Height_Ave_cm,Dry_Clover_g,Dry_Dead_g,Dry_Green_g,Dry_Total_g,GDM_g
0,train/ID1011485656.jpg,2015/9/4,Tas,Ryegrass_Clover,0.62,4.6667,0.0000,31.9984,16.2751,48.2735,16.2750
1,train/ID1012260530.jpg,2015/4/1,NSW,Lucerne,0.55,16.0000,0.0000,0.0000,7.6000,7.6000,7.6000
2,train/ID1025234388.jpg,2015/9/1,WA,SubcloverDalkeith,0.38,1.0000,6.0500,0.0000,0.0000,6.0500,6.0500
3,train/ID1028611175.jpg,2015/5/18,Tas,Ryegrass,0.66,5.0000,0.0000,30.9703,24.2376,55.2079,24.2376
4,train/ID1035947949.jpg,2015/9/11,Tas,Ryegrass,0.54,3.5000,0.4343,23.2239,10.5261,34.1844,10.9605
...,...,...,...,...,...,...,...,...,...,...,...
352,train/ID975115267.jpg,2015/7/8,WA,Clover,0.73,3.0000,40.0300,0.0000,0.8000,40.8300,40.8300
353,train/ID978026131.jpg,2015/9/4,Tas,Clover,0.83,3.1667,24.6445,4.1948,12.0601,40.8994,36.7046
354,train/ID980538882.jpg,2015/2/24,NSW,Phalaris,0.69,29.0000,0.0000,1.1457,91.6543,92.8000,91.6543
355,train/ID980878870.jpg,2015/7/8,WA,Clover,0.74,2.0000,32.3575,0.0000,2.0325,34.3900,34.3900


In [9]:
train_df['State'].value_counts()

State
Tas    138
Vic    112
NSW     75
WA      32
Name: count, dtype: int64

In [10]:
train_df.describe()

,Pre_GSHH_NDVI,Height_Ave_cm,Dry_Clover_g,Dry_Dead_g,Dry_Green_g,Dry_Total_g,GDM_g
count,357.000000,357.000000,357.000000,357.000000,357.000000,357.000000,357.000000
mean,0.657423,7.595985,6.649692,12.044548,26.624722,45.318097,33.274414
std,0.152142,10.285262,12.117761,12.402007,25.401232,27.984015,24.935822
min,0.160000,1.000000,0.000000,0.000000,0.000000,1.040000,1.040000
25%,0.560000,3.000000,0.000000,3.200000,8.800000,25.271500,16.026100
50%,0.690000,4.000000,1.423500,7.980900,20.800000,40.300000,27.108200
75%,0.770000,7.000000,7.242900,17.637800,35.083400,57.880000,43.675700
max,0.910000,70.000000,71.786500,83.840700,157.983600,185.700000,157.983600


In [11]:
IMG_SIZE = 224
N_SPLITS = 5
RANDOM_STATE = 42
BATCH_SIZE = 16
NUM_EPOCHS = 15  # можна почати з 5 для швидкого прогону
LR = 1e-3
WEIGHT_DECAY = 1e-4
USE_LOG_TARGETS = True

In [12]:
if USE_LOG_TARGETS:
    for col in TARGET_NAMES:
        train_df[col + "_log"] = np.log1p(train_df[col].clip(lower=0))
    TARGET_COLS = [c + "_log" for c in TARGET_NAMES]
else:
    TARGET_COLS = TARGET_NAMES

train_df[TARGET_COLS].describe()


,Dry_Green_g_log,Dry_Dead_g_log,Dry_Clover_g_log,GDM_g_log,Dry_Total_g_log
count,357.000000,357.000000,357.000000,357.000000,357.000000
mean,2.807202,2.087335,1.174004,3.273909,3.651970
std,1.186633,1.076524,1.246057,0.763890,0.643380
min,0.000000,0.000000,0.000000,0.712950,0.712950
25%,2.282382,1.435085,0.000000,2.834747,3.268485
50%,3.081910,2.195100,0.885213,3.336061,3.720862
75%,3.585833,2.925192,2.109352,3.799430,4.075501
max,5.068801,4.440775,4.287530,5.068801,5.229503


# Stratification Dry_Total_g

In [13]:
n_bins = N_SPLITS  

train_df["cv_strata"] = pd.qcut(
    train_df["Dry_Total_g"],
    q=n_bins,
    labels=False,
    duplicates="drop",
)

train_df["cv_strata"].value_counts()


cv_strata
0    72
4    72
3    71
2    71
1    71
Name: count, dtype: int64

# Augmentation

In [14]:
train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomResizedCrop(
        IMG_SIZE,
        scale=(0.8, 1.0),
        ratio=(0.9, 1.1),
    ),
    transforms.ToTensor(),
    # Нормалізація буде робитись в DINO-процесорі, тому тут НЕ нормалізуємо
])

val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
])


In [15]:
class PastureImageDataset(Dataset):
    def __init__(self, df, data_dir, transform, target_cols):
        self.df = df.reset_index(drop=True)
        self.data_dir = data_dir
        self.transform = transform
        self.target_cols = target_cols

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        img_path = os.path.join(self.data_dir, row["image_path"])
        image = Image.open(img_path).convert("RGB")

        if self.transform is not None:
            image = self.transform(image)  # тензор 3×H×W

        targets = torch.tensor(
            row[self.target_cols].values.astype("float32"),
            dtype=torch.float32
        )

        return {
            "image": image,
            "targets": targets,
            "image_path": row["image_path"],
        }


In [16]:
class DINOv2ImageEncoder(nn.Module):
    def __init__(self, model_dir=DINO_MODEL_DIR):
        super().__init__()
        self.processor = AutoImageProcessor.from_pretrained(model_dir)
        
        # ДУЖЕ ВАЖЛИВО: ми вже маємо [0,1] із ToTensor, тому не треба ще раз скейлити
        self.processor.do_rescale = False

        self.model = AutoModel.from_pretrained(model_dir)
        for p in self.model.parameters():
            p.requires_grad = False

        self.out_dim = self.model.config.hidden_size

    def forward(self, images):
        # images: B×3×H×W, float32, 0–1
        inputs = self.processor(
            images=images,
            return_tensors="pt",
            do_rescale=False
        ).to(images.device)

        outputs = self.model(**inputs)
        if hasattr(outputs, "pooler_output") and outputs.pooler_output is not None:
            emb = outputs.pooler_output
        else:
            emb = outputs.last_hidden_state[:, 0]

        return emb


In [17]:
class BiomassRegressor(nn.Module):
    def __init__(self, image_encoder, hidden_dim=512, n_targets=5):
        super().__init__()
        self.image_encoder = image_encoder
        in_dim = image_encoder.out_dim

        # простий MLP head
        self.head = nn.Sequential(
            nn.Linear(in_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, n_targets),
        )

    def forward(self, images):
        img_emb = self.image_encoder(images)  # B×D_img
        out = self.head(img_emb)              # B×5
        return out


In [18]:
def compute_loss(preds, targets):
    return nn.functional.mse_loss(preds, targets)

def compute_weighted_r2(y_true, y_pred):
    weights = np.array([0.1, 0.1, 0.1, 0.2, 0.5], dtype=np.float32)
    r2s = []
    for i in range(5):
        r2 = r2_score(y_true[:, i], y_pred[:, i])
        r2s.append(r2)
    r2s = np.array(r2s)
    return np.sum(weights * r2s)


In [19]:
def train_one_epoch(model, loader, optimizer):
    model.train()
    total_loss = 0.0

    for batch in loader:
        images = batch["image"].to(DEVICE)
        targets = batch["targets"].to(DEVICE)

        optimizer.zero_grad()
        preds = model(images)

        loss = compute_loss(preds, targets)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * images.size(0)

    return total_loss / len(loader.dataset)


@torch.no_grad()
def validate_one_epoch(model, loader):
    model.eval()
    total_loss = 0.0

    all_true = []
    all_pred = []

    for batch in loader:
        images = batch["image"].to(DEVICE)
        targets = batch["targets"].to(DEVICE)

        preds = model(images)

        loss = compute_loss(preds, targets)
        total_loss += loss.item() * images.size(0)

        # збираємо для R²
        y_true = targets.cpu().numpy()
        y_pred = preds.cpu().numpy()

        if USE_LOG_TARGETS:
            y_true = np.expm1(y_true)
            y_pred = np.expm1(y_pred)

        all_true.append(y_true)
        all_pred.append(y_pred)

    all_true = np.concatenate(all_true, axis=0)
    all_pred = np.concatenate(all_pred, axis=0)

    weighted_r2 = compute_weighted_r2(all_true, all_pred)
    avg_loss = total_loss / len(loader.dataset)

    return avg_loss, weighted_r2


In [20]:
skf = StratifiedKFold(
    n_splits=N_SPLITS,
    shuffle=True,
    random_state=RANDOM_STATE,
)

oof_true = []
oof_pred = []
oof_idx = []
fold_results = []

fold_states = []   # <-- сюди складатимемо best_state кожного фолда

In [21]:
project_name = "csiro-biomass-dino"
DINO_MODEL_DIR = "/kaggle/input/dinov2/pytorch/base/1"  # або як у тебе називається

In [22]:
for fold, (train_idx, val_idx) in enumerate(skf.split(train_df, train_df["cv_strata"])):
    print(f"\n=== Fold {fold} ===")
    df_tr = train_df.iloc[train_idx].reset_index(drop=True)
    df_va = train_df.iloc[val_idx].reset_index(drop=True)

    run = wandb.init(
        project=project_name,
        name=f"dino_fold_{fold}",
        config={
            "fold": fold,
            "img_size": IMG_SIZE,
            "batch_size": BATCH_SIZE,
            "epochs": NUM_EPOCHS,
            "lr": LR,
            "weight_decay": WEIGHT_DECAY,
            "use_log_targets": USE_LOG_TARGETS,
            "n_splits": N_SPLITS,
        },
        resume=None,    # дозволяє створювати багато run'ів в одному процесі
    )


    train_dataset = PastureImageDataset(
        df=df_tr,
        data_dir=DATA_DIR,
        transform=train_transform,
        target_cols=TARGET_COLS,
    )
    val_dataset = PastureImageDataset(
        df=df_va,
        data_dir=DATA_DIR,
        transform=val_transform,
        target_cols=TARGET_COLS,
    )

    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=2,
        pin_memory=True,
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=2,
        pin_memory=True,
    )

    # новий екземпляр моделі на кожен фолд
    image_encoder = DINOv2ImageEncoder(model_dir=DINO_MODEL_DIR)
    model = BiomassRegressor(
        image_encoder=image_encoder,
        hidden_dim=512,
        n_targets=len(TARGET_COLS),
    ).to(DEVICE)


    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=LR,
        weight_decay=WEIGHT_DECAY,
    )

    best_val_r2 = -1e9
    best_state = None

    for epoch in range(1, NUM_EPOCHS + 1):
        train_loss = train_one_epoch(model, train_loader, optimizer)
        val_loss, val_r2 = validate_one_epoch(model, val_loader)

        print(
            f"Fold {fold} | Epoch {epoch:02d} "
            f"| train_loss={train_loss:.4f} | val_loss={val_loss:.4f} | val_r2={val_r2:.4f}"
        )

        # 🔹 W&B: логування метрик
        wandb.log({
            "fold": fold,
            "epoch": epoch,
            "train_loss": train_loss,
            "val_loss": val_loss,
            "val_r2": val_r2,
        })

        # зберігаємо кращий стейт по R²
        if val_r2 > best_val_r2:
            best_val_r2 = val_r2
            best_state = {k: v.cpu() for k, v in model.state_dict().items()}

    print(f"Best val R² (fold {fold}) = {best_val_r2:.4f}")
    fold_results.append(best_val_r2)

    wandb.run.summary["best_val_r2"] = best_val_r2

    fold_states.append(best_state)

    # OOF-передбачення
    model.load_state_dict({k: v.to(DEVICE) for k, v in best_state.items()})
    model.eval()

    fold_true = []
    fold_pred = []

    with torch.no_grad():
        for batch in val_loader:
            images = batch["image"].to(DEVICE)
            targets = batch["targets"].to(DEVICE)
            preds = model(images)

            y_true = targets.cpu().numpy()
            y_pred = preds.cpu().numpy()

            if USE_LOG_TARGETS:
                y_true = np.expm1(y_true)
                y_pred = np.expm1(y_pred)

            fold_true.append(y_true)
            fold_pred.append(y_pred)

    fold_true = np.concatenate(fold_true, axis=0)
    fold_pred = np.concatenate(fold_pred, axis=0)

    oof_true.append(fold_true)
    oof_pred.append(fold_pred)
    oof_idx.append(val_idx)

    # чистимо пам'ять
    del model, image_encoder, optimizer, train_loader, val_loader
    gc.collect()
    torch.cuda.empty_cache()
    wandb.finish()




=== Fold 0 ===


wandb: Tracking run with wandb version 0.21.0
wandb: W&B syncing is set to `offline` in this directory. Run `wandb online` or set WANDB_MODE=online to enable cloud syncing.
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Fold 0 | Epoch 01 | train_loss=1.7501 | val_loss=0.7556 | val_r2=-0.9257
Fold 0 | Epoch 02 | train_loss=0.6188 | val_loss=0.5215 | val_r2=0.4854
Fold 0 | Epoch 03 | train_loss=0.4624 | val_loss=0.5568 | val_r2=0.4557
Fold 0 | Epoch 04 | train_loss=0.4265 | val_loss=0.5130 | val_r2=0.4040
Fold 0 | Epoch 05 | train_loss=0.3650 | val_loss=0.4935 | val_r2=0.4283
Fold 0 | Epoch 06 | train_loss=0.3346 | val_loss=0.4629 | val_r2=0.4187
Fold 0 | Epoch 07 | train_loss=0.3253 | val_loss=0.4472 | val_r2=0.5229
Fold 0 | Epoch 08 | train_loss=0.2929 | val_loss=0.5243 | val_r2=0.3168
Fold 0 | Epoch 09 | train_loss=0.2662 | val_loss=0.4877 | val_r2=0.4409
Fold 0 | Epoch 10 | train_loss=0.2660 | val_loss=0.4358 | val_r2=0.4943
Fold 0 | Epoch 11 | train_loss=0.2548 | val_loss=0.4522 | val_r2=0.4824
Fold 0 | Epoch 12 | train_loss=0.2317 | val_loss=0.4586 | val_r2=0.4930
Fold 0 | Epoch 13 | train_loss=0.2610 | val_loss=0.5126 | val_r2=0.4627
Fold 0 | Epoch 14 | train_loss=0.2468 | val_loss=0.4520 | val_r

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:      epoch ▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
wandb:       fold ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb: train_loss █▃▂▂▂▁▁▁▁▁▁▁▁▁▁
wandb:   val_loss █▃▄▃▂▂▁▃▂▁▁▁▃▁▁
wandb:     val_r2 ▁██▇█▇█▇██████▆
wandb: 
wandb: Run summary:
wandb: best_val_r2 0.52285
wandb:       epoch 15
wandb:        fold 0
wandb:  train_loss 0.24448
wandb:    val_loss 0.45368
wandb:      val_r2 0.14259
wandb: 
wandb: You can sync this run to the cloud by running:
wandb: wandb sync /kaggle/working/wandb/offline-run-20251207_142418-l8qjnf2y
wandb: Find logs at: ./wandb/offline-run-20251207_142418-l8qjnf2y/logs
wandb: Tracking run with wandb version 0.21.0
wandb: W&B syncing is set to `offline` in this directory. Run `wandb online` or set WANDB_MODE=online to enable cloud syncing.



=== Fold 1 ===
Fold 1 | Epoch 01 | train_loss=1.7675 | val_loss=0.6314 | val_r2=-8.3319
Fold 1 | Epoch 02 | train_loss=0.5738 | val_loss=0.5058 | val_r2=0.3601
Fold 1 | Epoch 03 | train_loss=0.4373 | val_loss=0.4769 | val_r2=0.4062
Fold 1 | Epoch 04 | train_loss=0.3852 | val_loss=0.4382 | val_r2=0.4584
Fold 1 | Epoch 05 | train_loss=0.3476 | val_loss=0.4423 | val_r2=0.3123
Fold 1 | Epoch 06 | train_loss=0.3775 | val_loss=0.4331 | val_r2=0.3443
Fold 1 | Epoch 07 | train_loss=0.3287 | val_loss=0.4654 | val_r2=0.2367
Fold 1 | Epoch 08 | train_loss=0.2768 | val_loss=0.4121 | val_r2=0.4019
Fold 1 | Epoch 09 | train_loss=0.2766 | val_loss=0.4566 | val_r2=0.3518
Fold 1 | Epoch 10 | train_loss=0.2632 | val_loss=0.4541 | val_r2=0.1888
Fold 1 | Epoch 11 | train_loss=0.2492 | val_loss=0.4264 | val_r2=0.2705
Fold 1 | Epoch 12 | train_loss=0.2903 | val_loss=0.4271 | val_r2=0.0765
Fold 1 | Epoch 13 | train_loss=0.2599 | val_loss=0.5456 | val_r2=0.2623
Fold 1 | Epoch 14 | train_loss=0.2260 | val_los

...

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:      epoch ▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
wandb:       fold ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb: train_loss █▃▂▂▂▂▂▁▁▁▁▁▁▁▁
wandb:   val_loss █▄▃▄▃▃▃▂▃▂▁▁▁▁▁
wandb:     val_r2 ▁▇█▆███████████
wandb: 
wandb: Run summary:
wandb: best_val_r2 0.47754
wandb:       epoch 15
wandb:        fold 2
wandb:  train_loss 0.24125
wandb:    val_loss 0.33368
wandb:      val_r2 0.43825
wandb: 
wandb: You can sync this run to the cloud by running:
wandb: wandb sync /kaggle/working/wandb/offline-run-20251207_143113-pbfp6hev
wandb: Find logs at: ./wandb/offline-run-20251207_143113-pbfp6hev/logs
wandb: Tracking run with wandb version 0.21.0
wandb: W&B syncing is set to `offline` in this directory. Run `wandb online` or set WANDB_MODE=online to enable cloud syncing.



=== Fold 3 ===
Fold 3 | Epoch 01 | train_loss=1.7058 | val_loss=0.7115 | val_r2=-0.6361
Fold 3 | Epoch 02 | train_loss=0.6036 | val_loss=0.5075 | val_r2=0.3239
Fold 3 | Epoch 03 | train_loss=0.4992 | val_loss=0.5462 | val_r2=0.2842
Fold 3 | Epoch 04 | train_loss=0.4069 | val_loss=0.4528 | val_r2=0.3386
Fold 3 | Epoch 05 | train_loss=0.3615 | val_loss=0.4567 | val_r2=0.3092
Fold 3 | Epoch 06 | train_loss=0.3449 | val_loss=0.4279 | val_r2=0.2924
Fold 3 | Epoch 07 | train_loss=0.3338 | val_loss=0.4220 | val_r2=0.2572
Fold 3 | Epoch 08 | train_loss=0.2982 | val_loss=0.4445 | val_r2=0.3659
Fold 3 | Epoch 09 | train_loss=0.3095 | val_loss=0.4332 | val_r2=0.4087
Fold 3 | Epoch 10 | train_loss=0.2889 | val_loss=0.4537 | val_r2=0.3791
Fold 3 | Epoch 11 | train_loss=0.2437 | val_loss=0.4423 | val_r2=0.3557
Fold 3 | Epoch 12 | train_loss=0.2495 | val_loss=0.4478 | val_r2=0.3694
Fold 3 | Epoch 13 | train_loss=0.2364 | val_loss=0.4452 | val_r2=0.4046
Fold 3 | Epoch 14 | train_loss=0.2607 | val_los

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:      epoch ▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
wandb:       fold ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb: train_loss █▃▂▂▂▂▁▁▁▁▁▁▁▁▁
wandb:   val_loss █▃▄▂▂▂▂▂▂▂▂▂▂▁▁
wandb:     val_r2 ▁▇▇▇▇▇▇▇█▇▇▇███
wandb: 
wandb: Run summary:
wandb: best_val_r2 0.484
wandb:       epoch 15
wandb:        fold 3
wandb:  train_loss 0.23593
wandb:    val_loss 0.39852
wandb:      val_r2 0.45991
wandb: 
wandb: You can sync this run to the cloud by running:
wandb: wandb sync /kaggle/working/wandb/offline-run-20251207_143438-n3uczixo
wandb: Find logs at: ./wandb/offline-run-20251207_143438-n3uczixo/logs
wandb: Tracking run with wandb version 0.21.0
wandb: W&B syncing is set to `offline` in this directory. Run `wandb online` or set WANDB_MODE=online to enable cloud syncing.



=== Fold 4 ===
Fold 4 | Epoch 01 | train_loss=1.9634 | val_loss=0.7800 | val_r2=-3.9731
Fold 4 | Epoch 02 | train_loss=0.7377 | val_loss=0.4653 | val_r2=-0.1314
Fold 4 | Epoch 03 | train_loss=0.5327 | val_loss=0.4317 | val_r2=0.4830
Fold 4 | Epoch 04 | train_loss=0.4557 | val_loss=0.3740 | val_r2=0.5343
Fold 4 | Epoch 05 | train_loss=0.3892 | val_loss=0.3846 | val_r2=0.4916
Fold 4 | Epoch 06 | train_loss=0.3506 | val_loss=0.3672 | val_r2=0.4973
Fold 4 | Epoch 07 | train_loss=0.3370 | val_loss=0.3794 | val_r2=0.4557
Fold 4 | Epoch 08 | train_loss=0.3173 | val_loss=0.3934 | val_r2=0.5082
Fold 4 | Epoch 09 | train_loss=0.3133 | val_loss=0.4378 | val_r2=0.3887
Fold 4 | Epoch 10 | train_loss=0.2762 | val_loss=0.3720 | val_r2=0.4182
Fold 4 | Epoch 11 | train_loss=0.2804 | val_loss=0.3780 | val_r2=0.4980
Fold 4 | Epoch 12 | train_loss=0.2801 | val_loss=0.3955 | val_r2=0.4652
Fold 4 | Epoch 13 | train_loss=0.2599 | val_loss=0.4076 | val_r2=0.1700
Fold 4 | Epoch 14 | train_loss=0.2507 | val_lo

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:      epoch ▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
wandb:       fold ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb: train_loss █▃▂▂▂▁▁▁▁▁▁▁▁▁▁
wandb:   val_loss █▃▂▁▁▁▁▂▂▁▁▂▂▁▂
wandb:     val_r2 ▁▇██████████▇██
wandb: 
wandb: Run summary:
wandb: best_val_r2 0.56924
wandb:       epoch 15
wandb:        fold 4
wandb:  train_loss 0.22789
wandb:    val_loss 0.40569
wandb:      val_r2 0.468
wandb: 
wandb: You can sync this run to the cloud by running:
wandb: wandb sync /kaggle/working/wandb/offline-run-20251207_143804-5v7wqh0h
wandb: Find logs at: ./wandb/offline-run-20251207_143804-5v7wqh0h/logs


In [23]:
oof_true = np.concatenate(oof_true, axis=0)
oof_pred = np.concatenate(oof_pred, axis=0)
oof_idx = np.concatenate(oof_idx, axis=0)

# перебудуємо назад в правильний порядок (як в train_df)
order = np.argsort(oof_idx)
oof_true = oof_true[order]
oof_pred = oof_pred[order]

overall_r2 = compute_weighted_r2(oof_true, oof_pred)
print("\nFold R²:", fold_results)
print("OOF weighted R²:", overall_r2)



Fold R²: [0.5228535349884507, 0.4583712869499014, 0.47753899727316784, 0.4840001492605024, 0.5692422175396034]
OOF weighted R²: 0.5012445584961676


In [24]:
test_long = pd.read_csv(os.path.join(DATA_DIR, "test.csv"))
print(test_long.head())

                    sample_id             image_path   target_name
0  ID1001187975__Dry_Clover_g  test/ID1001187975.jpg  Dry_Clover_g
1    ID1001187975__Dry_Dead_g  test/ID1001187975.jpg    Dry_Dead_g
2   ID1001187975__Dry_Green_g  test/ID1001187975.jpg   Dry_Green_g
3   ID1001187975__Dry_Total_g  test/ID1001187975.jpg   Dry_Total_g
4         ID1001187975__GDM_g  test/ID1001187975.jpg         GDM_g


In [25]:
test_img_df = (
    test_long[["image_path"]]
    .drop_duplicates()
    .reset_index(drop=True)
)

print("Num test images:", len(test_img_df))


Num test images: 1


In [26]:
class TestImageDataset(Dataset):
    def __init__(self, df, data_dir, transform):
        self.df = df.reset_index(drop=True)
        self.data_dir = data_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.data_dir, row["image_path"])
        image = Image.open(img_path).convert("RGB")

        if self.transform is not None:
            image = self.transform(image)

        return {
            "image": image,
            "image_path": row["image_path"],
        }

test_dataset = TestImageDataset(
    df=test_img_df,
    data_dir=DATA_DIR,
    transform=val_transform,   # як на валідації
)

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2,
    pin_memory=True,
)


In [27]:
all_fold_preds = []  # список [num_test_images × 5] по фолдах

for fold, state in enumerate(fold_states):
    print(f"Predicting on test with fold {fold} model...")

    image_encoder = DINOv2ImageEncoder(model_dir=DINO_MODEL_DIR)
    model = BiomassRegressor(
        image_encoder=image_encoder,
        hidden_dim=512,
        n_targets=len(TARGET_COLS),
    ).to(DEVICE)

    # завантажуємо кращі ваги
    model.load_state_dict({k: v.to(DEVICE) for k, v in state.items()})
    model.eval()

    fold_preds = []

    with torch.no_grad():
        for batch in test_loader:
            images = batch["image"].to(DEVICE)

            preds = model(images)  
            preds = preds.cpu().numpy()
            fold_preds.append(preds)

    fold_preds = np.concatenate(fold_preds, axis=0)
    all_fold_preds.append(fold_preds)

    # чистимо
    del model, image_encoder
    gc.collect()
    torch.cuda.empty_cache()


Predicting on test with fold 0 model...
Predicting on test with fold 1 model...
Predicting on test with fold 2 model...
Predicting on test with fold 3 model...
Predicting on test with fold 4 model...


In [28]:
# shape: (n_folds, num_test_images, 5) → усереднюємо по осі 0
all_fold_preds = np.stack(all_fold_preds, axis=0)
test_preds = all_fold_preds.mean(axis=0)  

# якщо вчилися на лог-таргетах — повертаємося в оригінальну шкалу
if USE_LOG_TARGETS:
    test_preds = np.expm1(test_preds)


In [29]:
image_to_idx = {
    path: idx
    for idx, path in enumerate(test_img_df["image_path"].values)
}

In [30]:
target_values = []

for _, row in test_long.iterrows():
    img_idx = image_to_idx[row["image_path"]]
    tname = row["target_name"]    

    tpos = TARGET_NAMES.index(tname)

    value = test_preds[img_idx, tpos]
    target_values.append(value)


In [31]:
submission = pd.DataFrame({
    "sample_id": test_long["sample_id"],
    "target": target_values,
})

submission.to_csv("submission.csv", index=False)
print(submission.head())


                    sample_id     target
0  ID1001187975__Dry_Clover_g   0.074653
1    ID1001187975__Dry_Dead_g  41.596066
2   ID1001187975__Dry_Green_g  37.508938
3   ID1001187975__Dry_Total_g  68.856308
4         ID1001187975__GDM_g  37.733761
